In [1]:
!pip install transformers
!pip install question_generation
!pip install torch

     |████████████████████████████████| 4.2 MB 14.8 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 22.2 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 136 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 77.4 MB/s 
     |████████████████████████████████| 271 kB 55.1 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("algolet/mt5-base-chinese-qg")

model = AutoModelForSeq2SeqLM.from_pretrained("algolet/mt5-base-chinese-qg")

from question_generation import pipeline

# gpu版本
qg = pipeline("question-generation", device="cuda") 
qa = pipeline("question-answering", device="cuda") 

Downloading:   0%|          | 0.00/428 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.93M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

In [3]:
import pandas as pd
import glob
import json
import csv
import re
import os
import time

path = r'./drive/MyDrive/Database/test_data/20220516-gpu' # use your path


all_files = glob.glob(path + "/*.csv")

start_time = time.time()
print('start_time: ', start_time)

for filename in all_files:
    test_list = []
    # 提取需要生成问题的文件名
    fname = filename.split('/')[-1:][0]
    fname = os.path.splitext(fname)[0]
    # 遍历词条信息进行问题生成
    with open(filename, 'r', encoding='utf-8-sig') as csvfile:
        csvReader = csv.DictReader(csvfile)
        i = 0
        for row in csvReader:
            y = {
                '_id':'',
                'text':'',
                'qa_pair':[]}
            i = i + 1
            text1 = row['实体']
            text2 = row['属性']
            text3 = row['值']
            full_text = text1 + ', '+ text2 +  ', '+  text3
            # 正则表达式，去除[]、()中的附加文本内容，减少对模型生成问题的无关影响
            full_text = re.sub("[\(\[].*?[\)\]]", "", full_text)
            # 对row进行转换，把原本的OrderedDict转换成string，此处因为保存原格式模型的表现更好，故此处进行保留
            # row = str(dict(row))
            print('FullText: ', full_text)
            y['_id'] = i
            questions = qg(full_text)
            j = 0
            for q in questions:
                z = {
                     'c_id':'',
                     'q':'',
                     'a':'',
                     'label':''
                }
                y['text'] = full_text
                answer = qa({
                    'question': str(q),
                    'context': full_text,
                })
                print('Q: ', q)
                # print('Row: ', row)
                if bool(answer) is False:
                    print('{ }')
                else:
                    j = j + 1
                    z['c_id'] = j
                    z['q'] = q
                    z['a'] = answer['answer']
                    print('Answer: ', answer['answer'])
                    print('\n')
                    y['qa_pair'].append(z)
            # 在满足q_a pair不空的情况下才进行添加
            if len(y['qa_pair']) == 0:
                pass
            else:
                test_list.append(y)
    # 对单个文件生成完后进行添加操作
    with open('./drive/MyDrive/Database/results/20220516-gpu/'+fname+'.json', 'a', encoding='utf-8-sig') as file:
        json.dump(test_list, file, ensure_ascii=False) 

end_time = time.time()
print('end_time: ', end_time)
use_time = end_time - start_time
print('use time: ', use_time)

start_time:  1652688388.1094487
FullText:  敦刻尔克大撤退, 地点, 法国敦刻尔克
Q:  敦刻尔克大撤退地点是哪里?
Answer:  敦刻尔克


Q:  大撤退的地点在哪里?
{ }
Q:  法国大撤退位于哪里?
Answer:  敦刻尔克


FullText:  敦刻尔克大撤退, 时间, 1940年5月26日 - 1940年6月4日
Q:  敦刻尔克什么时候撤退的?
Answer:  1940


Q:  哪一个城市在1940年撤退?
Answer:  敦刻尔克


Q:  在1940年5月26日,哪一个城市撤退了?
Answer:  敦刻尔克


FullText:  敦刻尔克大撤退, 参战方, 英国，法国，比利时，德国
Q:  敦刻尔克的参战方是哪个国家?
Answer:  英国


Q:  哪个国家打败了敦刻尔科?
Answer:  比利时


Q:  哪支球队在敦刻尔顿大撤退?
Answer:  英国，法国，比利时


FullText:  敦刻尔克大撤退, 结果, 英法联军成功撤退至英国本土
Q:  英法联军从敦刻尔克撤退到哪个国家?
Answer:  英国


Q:  英国的哪个城市撤退了?
Answer:  敦刻尔克


Q:  哪个国家撤退了敦刻尔科?
Answer:  英国


FullText:  敦刻尔克大撤退, 参战方兵力, 英法联军 约400,000人
德军 约800,000人
Q:  敦刻尔克撤退后,英法联军有多少人参战?
Answer:  400,000人


Q:  英法联军队有多少人参加战斗?
Answer:  400,000人


Q:  德军有多少人参加战役?
Answer:  800,000人


FullText:  敦刻尔克大撤退, 伤亡情况, 盟军 2.8万余人阵亡，4万人被俘
德军 1万余人阵亡
Q:  敦刻尔克大撤退后有多少人被俘?
Answer:  4万


Q:  德军有多少人阵亡?
Answer:  1万余人


Q:  盟军有多少伤亡?
Answer:  2.8万


FullText:  敦刻尔克大撤退, 主要指挥官, 罗德·高特，魏刚，伦德施泰特
Q:  谁是敦刻尔克大撤退的指挥官?
Answer:  罗德·高特


Q:  谁是罗德·高特,魏刚和伦德施